# MongoDB

## What is MongoDB?

**MongoDB** is a document-oriented non-relational database manager that makes use of key/value orientation. Its name comes from humongous and it uses the BSON (compiled JSON) format to store data. It is one of the best known database managers, on a par with MySQL. Some concepts associated with this type of databases are:

* JSON
* Document
* Collection

Advantages:

* Multiplatform
* High performance
* High Availability
* Easy scalability

Some analogous concepts to SQL:

* Table -> Collection

* Register -> Document: a set of key-value pairs

* Column -> Key

![Dcoker Architecture](Images/Module5/mongo_server.PNG)

## Install MongoDB Community Server
1. Download and install **_MongoDB Community Server_** from [link](https://www.mongodb.com/try/download/community):
2. In the disk where you installed **_MongoDB Community Server_** create de directory **data/db**

## Tutorial MongoDB

1. Go to MongoDB directory, usually is somethiing like **./MongoDB/Server/5.0/bin**
2. Open mongod.exe(This is the server)
3. Open mongo.exe(This is the client)
4. Try mongo:
```bash
 use mydb # Switch to mydb data base if it does'nt exist it will be create it 
 db # Show in wich databe we are
 show dbs # Show all data bases
 db.users.insert({"Chris": 23}) # Create a collection named "users" wih the document {"Chris": 23}
```

## What is PyMongo?
PyMongo is an intermediary between Python and MongoDB. PyMongo is a Python distribution containing tools for working with MongoDB, and is the recommended way to work with MongoDB from Python.

## Tutorial PyMongo 

[Source](https://pymongo.readthedocs.io/en/stable/tutorial.html)

This tutorial is intended as an introduction to working with MongoDB and PyMongo. 

First you need to install PyMongo ```pip install pymongo```, this tutorial also assumes that a MongoDB instance is running on the default host and port. Assuming you have downloaded and installed MongoDB, you can start it like so:

```bash
mongod
```


**Note:** in windows you need to execute mongod.exe


### Making a Connection with MongoClient

The first step when working with **PyMongo** is to create a `MongoClient` to the running **mongod** instance. Doing so is easy:

```python
from pymongo import MongoClient
client = MongoClient()

```

The above code will connect on the default host and port. We can also specify the host and port explicitly, as follows:

```python
client = MongoClient('localhost', 27017)
```

Or use the MongoDB URI format:

```python
client = MongoClient('mongodb://localhost:27017/')
```

### Getting a Database

A single instance of MongoDB can support multiple independent [databases](https://www.mongodb.com/docs/manual/core/databases-and-collections). When working with PyMongo you access databases using attribute style access on `MongoClient` instances:

```python
db = client.test_database
```

If your database name is such that using attribute style access won’t work (like `test-database`), you can use dictionary style access instead:

```python
db = client['test-database']
```

If you want to list of the databases in the client:

```python
print(client.list_database_names())
```


### Getting a Collection

A [collection](https://www.mongodb.com/docs/manual/core/databases-and-collections) is a group of documents stored in MongoDB, and can be thought of as roughly the equivalent of a table in a relational database. Getting a collection in PyMongo works the same as getting a database:

```python
collection = db.test_collection
```

or (using dictionary style access):

```python
collection = db['test-collection']
```

An important note about collections (and databases) in MongoDB is that they are created lazily - none of the above commands have actually performed any operations on the MongoDB server. Collections and databases are created when the first document is inserted into them.


### Documents

Data in MongoDB is represented (and stored) using JSON-style documents. In PyMongo we use dictionaries to represent documents. As an example, the following dictionary might be used to represent a blog post:

```python
import datetime
post = {"author": "Mike",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}
```

Note that documents can contain native Python types (like [`datetime.datetime`](https://docs.python.org/3/library/datetime.html#datetime.datetime "(in Python v3.10)") instances) which will be automatically converted to and from the appropriate [BSON](https://bsonspec.org/) types.



### Inserting a Document


To insert a document into a collection we can use the [`insert_one()`](api/pymongo/collection.html#pymongo.collection.Collection.insert_one "pymongo.collection.Collection.insert_one") method:

```python
posts = db.posts
post_id = posts.insert_one(post).inserted_id

post_id
```




When a document is inserted a special key, `"_id"`, is automatically added if the document doesn’t already contain an `"_id"` key. The value of `"_id"` must be unique across the collection. [`insert_one()`](api/pymongo/collection.html#pymongo.collection.Collection.insert_one "pymongo.collection.Collection.insert_one") returns an instance of [`InsertOneResult`](api/pymongo/results.html#pymongo.results.InsertOneResult "pymongo.results.InsertOneResult"). For more information on `"_id"`, see the [documentation on \_id](https://www.mongodb.com/docs/manual/reference/method/ObjectId/).

After inserting the first document, the _posts_ collection has actually been created on the server. We can verify this by listing all of the collections in our database:

```python
db.list_collection_names() 

['posts']
```


### Getting a Single Document With `find_one()`

The most basic type of query that can be performed in MongoDB is [`find_one()`](api/pymongo/collection.html#pymongo.collection.Collection.find_one "pymongo.collection.Collection.find_one"). This method returns a single document matching a query (or `None` if there are no matches). It is useful when you know there is only one matching document, or are only interested in the first match. Here we use [`find_one()`](api/pymongo/collection.html#pymongo.collection.Collection.find_one "pymongo.collection.Collection.find_one") to get the first document from the posts collection:

```python
import pprint
pprint.pprint(posts.find_one())

{'_id': ObjectId('...'),
 'author': 'Mike',
 'date': datetime.datetime(...),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
```


The result is a dictionary matching the one that we inserted previously.

**Note:** The returned document contains an `"_id"`, which was automatically added on insert.

[`find_one()`](api/pymongo/collection.html#pymongo.collection.Collection.find_one "pymongo.collection.Collection.find_one") also supports querying on specific elements that the resulting document must match. To limit our results to a document with author “Mike” we do:
```python
pprint.pprint(posts.find_one({"author": "Mike"}))

{'_id': ObjectId('...'),
 'author': 'Mike',
 'date': datetime.datetime(...),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
```



If we try with a different author, like “Eliot”, we’ll get no result:
```python
posts.find_one({"author": "Eliot"})
```

### Querying By ObjectId


We can also find a post by its `_id`, which in our example is an ObjectId:

```python
post_id

ObjectId(...)
```

```python
pprint.pprint(posts.find_one({"_id": post_id}))

{'_id': ObjectId('...'),
 'author': 'Mike',
 'date': datetime.datetime(...),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
```


Note that an ObjectId is not the same as its string representation:

```python
post_id_as_str = str(post_id)
posts.find_one({"_id": post_id_as_str}) # No result
```

A common task in web applications is to get an ObjectId from the request URL and find the matching document. It’s necessary in this case to **convert the ObjectId from a string** before passing it to `find_one`:

```python
from bson.objectid import ObjectId

# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})
```
See also

[When I query for a document by ObjectId in my web application I get no result](https://pymongo.readthedocs.io/en/stable/faq.html#web-application-querying-by-objectid)


If you want to know more about PyMongo go to [PyMongo-Tutorial](https://pymongo.readthedocs.io/en/stable/tutorial.html)


## References

https://www.youtube.com/watch?v=c8n6JsQuX2A

https://pymongo.readthedocs.io/en/stable/tutorial.html

https://github.com/beduExpert/A1-Introduccion-a-Bases-de-Datos-Santander2021

In [ ]:
Falta agregar Source and [Back to the top](#)